In [9]:
import os

In [6]:
from selenium.webdriver.common.by import By

In [7]:
import time

In [8]:
import requests

In [10]:
import io

In [10]:
from PIL import Image

In [11]:
import hashlib

In [16]:
from selenium import webdriver
DRIVER_PATH = 'C:/Program Files/chromedriver_win32/chromedriver.exe'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

C:\Users\arviv\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
wd.get('https://google.com')

In [17]:
wd.get("https://drive.google.com/drive/folders/1yw8vIWUSqZKvj-pNG8Yn_pYpj8R1iuo6")

In [6]:
search_box = wd.find_element(By.CSS_SELECTOR,'input.gLFyf')
search_box.send_keys('Dogs')

In [18]:
def scroll_to_end(wd):
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
scroll_to_end(wd)

In [2]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements(By.CSS_SELECTOR,"img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements(By.CSS_SELECTOR,'img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element(By.CSS_SELECTOR,".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [3]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [27]:
fetch_image_urls('dogs',5,wd)

Found: 48 search results. Extracting links from 0:48
Found: 1 image links, looking for more ...


In [7]:
def search_and_download(search_term:str,driver_path:str,target_path='D:/images',number_images=100):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [46]:
food_item="Chicken Steak"

In [47]:
search_and_download(food_item,DRIVER_PATH)

C:\Users\arviv\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  import sys


Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
SUCCESS - saved https://www.recipetineats.com/wp-content/uploads/2021/07/Mushroom-Stuffed-Chicken-Breast_14-SQ.jpg - as D:/images\chicken_steak\684736b165.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTpItzAfgW-G32C9aBcjJBMGVqALygLKK1Blw&usqp=CAU - as D:/images\chicken_steak\cff30a2541.jpg
SUCCESS - saved https://i.ytimg.com/vi/CbbmFlsecLI/maxresdefault.jpg - as D:/images\chicken_steak\0862975aa5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT_1me4W-QTZ29_kOQhFxT4pTNczS3RLfq0Eg&usqp=CAU - as D:/images\chicken_steak\76979f7085.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQNMEbXmnVdcWJSTFJH131jet7k8KGW99Wc7A&usqp=CAU - as D:/images\chicken_steak\3526d563d6.jpg
SUCCESS - saved https://www.aheadofthyme.com/wp-content/uploads/2022/04/mexican-chicken-breast-2.jpg - as D:/images\chicken_steak\d4300fffb2.jpg
SUCCESS - saved ht

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSGLpuEbzpQsgKJe44N5YmXXuwbXEUgnVGsbA&usqp=CAU - as D:/images\chicken_steak\94c7571fc6.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTN7WcVNWoLzbuIX7TjV09-2KVv-jCOqk5FOQ&usqp=CAU - as D:/images\chicken_steak\7b80637671.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ5iJPcPxchTfeIRpijQMFbO8W4fEnUBiw8cQ&usqp=CAU - as D:/images\chicken_steak\58fdf204dd.jpg
SUCCESS - saved https://www.thecookierookie.com/wp-content/uploads/2021/06/featured-chicken-fried-steak-recipe.jpg - as D:/images\chicken_steak\07228ec3b4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS3XW4uOVA_DwtQKp8vGF7ZqnniaefhknS2hw&usqp=CAU - as D:/images\chicken_steak\2dc7e48e9d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTwILKlkoZM4O_pHMO0uHinVJJdM-CWwKA-jA&usqp=CAU - as D:/images\chicken_steak\23e789115c.jpg
SUCCESS - saved https://img.lovepik.com/f